In [1]:
# Interestingly Logistic Regression in performing well on the data with TF-IDF Vectorizer.
# There were many big models like XgBoost , RF , GB but Logistic Regression is performing to well.
# For this moment of time we are focusing on learning to integrate the concepts learned in workflow so not focusing on accuracy much.
# I am taking Logistic as my model now for final experimentation.